In [1]:
from collections import namedtuple

In [21]:
Files = namedtuple('filelist',['categorical','date','numeric'])
Data = namedtuple('data',['categorical','date','numeric','outcome'])
train_files = Files('./Data/train_categorical.csv.gz',
                './Data/train_date.csv.gz',
                './Data/train_numeric.csv.gz')
test_files = Files('./Data/test_categorical.csv.gz',
                './Data/test_date.csv.gz',
                './Data/test_numeric.csv.gz')

In [22]:
# train  
train = Data(sc.textFile(train_files.categorical).cache(),
             sc.textFile(train_files.date).cache(),
             sc.textFile(train_files.numeric).cache())

# test 
test = Data(sc.textFile(test_files.categorical).cache(),
             sc.textFile(test_files.date).cache(),
             sc.textFile(test_files.numeric).cache())

TypeError: __new__() takes exactly 5 arguments (4 given)

In [25]:
# filter out first row
remove_header = lambda x: x.split(',')[0]!='Id'

# split key and fields
key_split = lambda x: (int(x.split(',')[0]), x.split(',')[1:])

# return outcome from numeric data
get_outcome = lambda (k,v): (k, int(v[-1]))

def convert_numeric(x):
    ''' Gather numeric features '''
    k,v = x
    output = [0]*len(v[:-1])
    for i,x in enumerate(v[:-1]):
        output[i] = float(x) if len(x)>0 else 0
    return (int(k), output)

train_filtered = Data(
    train.categorical.filter(remove_header).map(key_split).repartition(100),
    train.date.filter(remove_header).map(key_split).repartition(100),
    train.numeric.filter(remove_header).map(key_split).map(convert_numeric).repartition(100),
    train.numeric.filter(remove_header).map(key_split).map(get_outcome).repartition(100)
)
test_filtered = Data(
    test.categorical.filter(remove_header).map(key_split).repartition(100),
    test.date.filter(remove_header).map(key_split).repartition(100),
    test.numeric.filter(remove_header).map(key_split).map(convert_numeric).repartition(100),
    test.numeric.filter(remove_header).map(key_split).map(get_outcome).repartition(100)
)

In [ ]:
train_filtered.outcome.first()